It shouldn't be a complicated extension, but I'm concerned about that huge number at the bottom. Let's do a brute force version first, and then see whether it's feasible in the time.

In [1]:
testInput_str='''
..#
#..
...
'''

Set this up in the same way as previously, but with additional keys to account for the weakened and flagged states:

In [2]:
infected_set=set()
input_ls=testInput_str.strip().split('\n')

for (j, row) in enumerate(input_ls):
    for (i, ch) in enumerate(row):
        if ch=='#':
            infected_set.add((i, j))

state_dict={'facing':'u', 'pos':(len(input_ls[0])//2, len(input_ls)//2),
            'infected':infected_set, 'weakened':set(), 'flagged':set()}

state_dict

{'facing': 'u',
 'flagged': set(),
 'infected': {(0, 1), (2, 0)},
 'pos': (1, 1),
 'weakened': set()}

OK, now define a function to do a burst:

In [3]:
def burst2(stateIn_dict):
    '''
    Carry out a single burst for the virus carrier.
    Returns a pair of the next state, and a bool 
    stating whether the current node was infected.
    '''
    infectOut_bool=False
    
    if stateIn_dict['pos'] in stateIn_dict['infected']:
        # If on an infected node, turn right
        facingOut_chr={'u':'r', 'r':'d', 'd':'l', 'l':'u'}[stateIn_dict['facing']]
    elif stateIn_dict['pos'] in stateIn_dict['weakened']:
        # If on a weakened node, carry on
        facingOut_chr=stateIn_dict['facing']
    elif stateIn_dict['pos'] in stateIn_dict['flagged']:
        # If on a flagged node, reverse
        facingOut_chr={'u':'d', 'd':'u', 'r':'l', 'l':'r'}[stateIn_dict['facing']]
    else:
        # Otherwise turn left:
        facingOut_chr={'u':'l', 'l':'d', 'd':'r', 'r':'u'}[stateIn_dict['facing']]
    
    # Modify the state of the node
    infectedOut_set=stateIn_dict['infected'].copy() # Shallow copy. Never mind...
    weakenedOut_set=stateIn_dict['weakened'].copy()
    flaggedOut_set=stateIn_dict['flagged'].copy()
    
    if stateIn_dict['pos'] in stateIn_dict['weakened']:
        # If weakened, infect:
        infectedOut_set.add(stateIn_dict['pos'])
        weakenedOut_set.remove(stateIn_dict['pos'])
        infectOut_bool=True
    elif stateIn_dict['pos'] in stateIn_dict['infected']:
        # If infected, flag:
        infectedOut_set.remove(stateIn_dict['pos'])
        flaggedOut_set.add(stateIn_dict['pos'])
    elif stateIn_dict['pos'] in stateIn_dict['flagged']:
        # If flagged, clean:
        flaggedOut_set.remove(stateIn_dict['pos'])
    else:
        # If clean, weaken
        weakenedOut_set.add(stateIn_dict['pos'])
    
    # And finally move:
    (rowOut_i, colOut_i)=stateIn_dict['pos']
    if facingOut_chr=='u':
        colOut_i -= 1
    elif facingOut_chr=='d':
        colOut_i += 1
    elif facingOut_chr=='l':
        rowOut_i -= 1
    elif facingOut_chr=='r':
        rowOut_i += 1
        
    return ({'facing':facingOut_chr, 
             'pos':(rowOut_i, colOut_i),
             'infected':infectedOut_set,
             'flagged':flaggedOut_set,
             'weakened':weakenedOut_set},
            infectOut_bool)

    

In [4]:
infected_set=set()
input_ls=testInput_str.strip().split('\n')

for (j, row) in enumerate(input_ls):
    for (i, ch) in enumerate(row):
        if ch=='#':
            infected_set.add((i, j))

state_dict={'facing':'u', 'pos':(len(input_ls[0])//2, len(input_ls)//2),
            'infected':infected_set, 'weakened':set(), 'flagged':set()}

state_dict

{'facing': 'u',
 'flagged': set(),
 'infected': {(0, 1), (2, 0)},
 'pos': (1, 1),
 'weakened': set()}

In [5]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'l', 'pos': (0, 1), 'infected': {(0, 1), (2, 0)}, 'flagged': set(), 'weakened': {(1, 1)}} False


In [6]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'u', 'pos': (0, 0), 'infected': {(2, 0)}, 'flagged': {(0, 1)}, 'weakened': {(1, 1)}} False


In [7]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'l', 'pos': (-1, 0), 'infected': {(2, 0)}, 'flagged': {(0, 1)}, 'weakened': {(0, 0), (1, 1)}} False


In [8]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'd', 'pos': (-1, 1), 'infected': {(2, 0)}, 'flagged': {(0, 1)}, 'weakened': {(-1, 0), (0, 0), (1, 1)}} False


In [9]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'r', 'pos': (0, 1), 'infected': {(2, 0)}, 'flagged': {(0, 1)}, 'weakened': {(-1, 1), (-1, 0), (0, 0), (1, 1)}} False


In [10]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'l', 'pos': (-1, 1), 'infected': {(2, 0)}, 'flagged': set(), 'weakened': {(-1, 1), (-1, 0), (0, 0), (1, 1)}} False


In [11]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'l', 'pos': (-2, 1), 'infected': {(2, 0), (-1, 1)}, 'flagged': set(), 'weakened': {(-1, 0), (0, 0), (1, 1)}} True


In [12]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'd', 'pos': (-2, 2), 'infected': {(2, 0), (-1, 1)}, 'flagged': set(), 'weakened': {(-2, 1), (0, 0), (-1, 0), (1, 1)}} False


In [13]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'r', 'pos': (-1, 2), 'infected': {(2, 0), (-1, 1)}, 'flagged': set(), 'weakened': {(-2, 1), (0, 0), (-1, 0), (-2, 2), (1, 1)}} False


In [14]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'u', 'pos': (-1, 1), 'infected': {(2, 0), (-1, 1)}, 'flagged': set(), 'weakened': {(-2, 1), (0, 0), (-1, 0), (-2, 2), (-1, 2), (1, 1)}} False


In [15]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'r', 'pos': (0, 1), 'infected': {(2, 0)}, 'flagged': {(-1, 1)}, 'weakened': {(-2, 1), (0, 0), (-1, 0), (-2, 2), (-1, 2), (1, 1)}} False


In [16]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'u', 'pos': (0, 0), 'infected': {(2, 0)}, 'flagged': {(-1, 1)}, 'weakened': {(0, 1), (-2, 1), (0, 0), (-1, 0), (-2, 2), (-1, 2), (1, 1)}} False


In [17]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'u', 'pos': (0, -1), 'infected': {(2, 0), (0, 0)}, 'flagged': {(-1, 1)}, 'weakened': {(0, 1), (-2, 1), (-1, 0), (-2, 2), (-1, 2), (1, 1)}} True


In [18]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'l', 'pos': (-1, -1), 'infected': {(2, 0), (0, 0)}, 'flagged': {(-1, 1)}, 'weakened': {(0, 1), (-2, 1), (-1, 0), (-2, 2), (-1, 2), (0, -1), (1, 1)}} False


In [19]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'd', 'pos': (-1, 0), 'infected': {(2, 0), (0, 0)}, 'flagged': {(-1, 1)}, 'weakened': {(0, 1), (-2, 1), (-1, 0), (-2, 2), (-1, 2), (-1, -1), (0, -1), (1, 1)}} False


In [20]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'd', 'pos': (-1, 1), 'infected': {(2, 0), (0, 0), (-1, 0)}, 'flagged': {(-1, 1)}, 'weakened': {(-2, 2), (-1, 2), (0, 1), (-2, 1), (-1, -1), (0, -1), (1, 1)}} True


In [21]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'u', 'pos': (-1, 0), 'infected': {(2, 0), (0, 0), (-1, 0)}, 'flagged': set(), 'weakened': {(0, 1), (-2, 1), (-2, 2), (-1, 2), (-1, -1), (0, -1), (1, 1)}} False


In [22]:
(state_dict, infected_bool)=burst2(state_dict)
print(state_dict, infected_bool)

{'facing': 'r', 'pos': (0, 0), 'infected': {(2, 0), (0, 0)}, 'flagged': {(-1, 0)}, 'weakened': {(0, 1), (-2, 1), (-2, 2), (-1, 2), (-1, -1), (0, -1), (1, 1)}} False


etc. Seems to be working OK. 

Well... I can either work out how to optimise this, or leave it running while I catch up on The Blue Planet. Fishes, here I come...

In [23]:
infected_set=set()
input_ls=testInput_str.strip().split('\n')

for (j, row) in enumerate(input_ls):
    for (i, ch) in enumerate(row):
        if ch=='#':
            infected_set.add((i, j))

state_dict={'facing':'u', 'pos':(len(input_ls[0])//2, len(input_ls)//2),
            'infected':infected_set, 'weakened':set(), 'flagged':set()}

infectingBursts_i=0
for i in range(100):
    (state_dict, infected_bool)=burst2(state_dict)
    if infected_bool:
        infectingBursts_i+=1

assert infectingBursts_i==26


In [24]:
infected_set=set()
input_ls=testInput_str.strip().split('\n')

for (j, row) in enumerate(input_ls):
    for (i, ch) in enumerate(row):
        if ch=='#':
            infected_set.add((i, j))

state_dict={'facing':'u', 'pos':(len(input_ls[0])//2, len(input_ls)//2),
            'infected':infected_set, 'weakened':set(), 'flagged':set()}

infectingBursts_i=0
for i in range(10000000):
    if not i%100000:
        print(i, end=' ')
    (state_dict, infected_bool)=burst2(state_dict)
    if infected_bool:
        infectingBursts_i+=1

print(infectingBursts_i)        

assert infectingBursts_i==2511944


0 100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 1100000 1200000 1300000 1400000 1500000 1600000 1700000 1800000 1900000 2000000 2100000 2200000 2300000 2400000 2500000 2600000 2700000 2800000 2900000 3000000 3100000 3200000 3300000 3400000 3500000 3600000 3700000 3800000 3900000 4000000 4100000 4200000 4300000 4400000 4500000 4600000 4700000 4800000 4900000 5000000 5100000 5200000 5300000 5400000 5500000 5600000 5700000 5800000 5900000 6000000 6100000 6200000 6300000 6400000 6500000 6600000 6700000 6800000 6900000 7000000 7100000 7200000 7300000 7400000 7500000 7600000 7700000 7800000 7900000 8000000 8100000 8200000 8300000 8400000 8500000 8600000 8700000 8800000 8900000 9000000 9100000 9200000 9300000 9400000 9500000 9600000 9700000 9800000 9900000 2511944


Good. Very slow but I'm not in a rush. So do the puzzle input:

In [26]:
with open('data/day22.txt') as fIn:
    puzzleInput_str=fIn.read()

infected_set=set()
input_ls=puzzleInput_str.strip().split('\n')

for (j, row) in enumerate(input_ls):
    for (i, ch) in enumerate(row):
        if ch=='#':
            infected_set.add((i, j))

state_dict={'facing':'u', 'pos':(len(input_ls[0])//2, len(input_ls)//2),
            'infected':infected_set, 'weakened':set(), 'flagged':set()}

infectingBursts_i=0
for i in range(10000000):
    if not i%100000:
        print(i, end=' ')
    (state_dict, infected_bool)=burst2(state_dict)
    if infected_bool:
        infectingBursts_i+=1


print(infectingBursts_i)        


0 100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 1100000 1200000 1300000 1400000 1500000 1600000 1700000 1800000 1900000 2000000 2100000 2200000 2300000 2400000 2500000 2600000 2700000 2800000 2900000 3000000 3100000 3200000 3300000 3400000 3500000 3600000 3700000 3800000 3900000 4000000 4100000 4200000 4300000 4400000 4500000 4600000 4700000 4800000 4900000 5000000 5100000 5200000 5300000 5400000 5500000 5600000 5700000 5800000 5900000 6000000 6100000 6200000 6300000 6400000 6500000 6600000 6700000 6800000 6900000 7000000 7100000 7200000 7300000 7400000 7500000 7600000 7700000 7800000 7900000 8000000 8100000 8200000 8300000 8400000 8500000 8600000 8700000 8800000 8900000 9000000 9100000 9200000 9300000 9400000 9500000 9600000 9700000 9800000 9900000 2511895
